## Model Training

In [52]:
import pandas as pd

In [53]:
df = pd.read_csv('./data/gemstone.csv')

df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [54]:
# Drop the id column.
df = df.drop(labels=['id'], axis = 1)

In [55]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [56]:
X = df.drop(labels=['price'], axis=1)
y = df[['price']]

In [57]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [58]:
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [59]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193573 entries, 0 to 193572
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   carat    193573 non-null  float64
 1   cut      193573 non-null  object 
 2   color    193573 non-null  object 
 3   clarity  193573 non-null  object 
 4   depth    193573 non-null  float64
 5   table    193573 non-null  float64
 6   x        193573 non-null  float64
 7   y        193573 non-null  float64
 8   z        193573 non-null  float64
dtypes: float64(6), object(3)
memory usage: 13.3+ MB


In [60]:
categorical_columns = X.select_dtypes(include='object').columns
numerical_columns = X.select_dtypes(exclude='object').columns

In [61]:
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [62]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [63]:
df['cut'].unique()

array(['Premium', 'Very Good', 'Ideal', 'Good', 'Fair'], dtype=object)

In [64]:
df['color'].unique()

array(['F', 'J', 'G', 'E', 'D', 'H', 'I'], dtype=object)

In [65]:
df['clarity'].unique()

array(['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1'],
      dtype=object)

In [66]:
cut_category = ['Premium', 'Very Good', 'Ideal', 'Good', 'Fair']
color_category = ['F', 'J', 'G', 'E', 'D', 'H', 'I']
clarity_category = ['VS2', 'SI2', 'VS1', 'SI1', 'IF', 'VVS2', 'VVS1', 'I1']

In [67]:
from sklearn.impute import SimpleImputer  # Handling missing values
from sklearn.preprocessing import StandardScaler # Handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # Encoding is for ordinal values for the categorical featres

In [68]:
## TO create pipeline we need to import these libraries

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [69]:
# Numerical pipeline

num_pipeline = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scalar', StandardScaler())
    ]
)

In [70]:
# Categorical pipeline

cat_pipeline = Pipeline(
    steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder', OrdinalEncoder(categories=[cut_category, color_category, clarity_category])),
        ('scalar', StandardScaler())

    ]

)

In [71]:
preprocessor = ColumnTransformer(
    [
        ('num_pipeline', num_pipeline, numerical_columns),
        ('cat_pipeline', cat_pipeline, categorical_columns)

    ]
)

In [72]:
## Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [73]:
preprocessor.fit_transform(X_train)

array([[-0.82314374, -1.12998781, -0.64189666, ...,  0.65242448,
         0.05875301,  1.60539392],
       [ 0.94502267, -1.77782269,  0.92190185, ..., -0.38150613,
         1.12307004, -0.07924587],
       [ 1.9584839 ,  0.16568195,  0.40063568, ..., -1.41543675,
         1.12307004, -1.20233906],
       ...,
       [ 0.92345966,  0.90606467,  0.40063568, ..., -1.41543675,
        -0.47340551, -1.20233906],
       [-1.03877378, -0.66724861, -0.64189666, ..., -0.38150613,
        -0.47340551,  2.16694052],
       [-1.03877378, -0.01941373,  0.92190185, ..., -0.38150613,
        -0.47340551, -0.64079246]])

In [74]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out())

In [75]:
X_test = pd.DataFrame(preprocessor.transform(X_test), columns=preprocessor.get_feature_names_out())

In [78]:
## Model train
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [79]:
linerR = LinearRegression()
linerR.fit(X_train, y_train)

LinearRegression()

In [80]:
linerR.coef_

array([[ 6538.82211476,  -232.4061404 ,  -177.05602534, -2090.53215704,
         -556.05883268,   -44.19321465,   -10.72128479,  -181.811724  ,
          121.17907723]])

In [81]:
linerR.intercept_

array([3976.8787389])

In [84]:
import numpy as np

def evaluate_model (true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_squre = r2_score(true, predicted)
    return mae, rmse, r2_squre

In [101]:
## Train multiple models 
models = {
    'LinearRegression':LinearRegression(),
    'LassoRegression':Lasso(),
    'RidgeRegression':Ridge(),
    'ElasticNet':ElasticNet()
}

In [104]:
model_list = []
r2_list = []

traind_model_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    # Make predections
    y_pred = model.predict(X_test)
    
    mae, rmse, r2_squre = evaluate_model(y_test, y_pred)

    print (list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model training performance:")
    print("RMSE ", rmse)
    print("MAE: ", mae)
    print("R2 Score: ", r2_squre * 10)
    print("\n")
    print("=" * 35)

LinearRegression
Model training performance:
RMSE  1196.824881597249
MAE:  723.6034569047462
R2 Score:  9.11356057138532




LassoRegression
Model training performance:
RMSE  1196.793686086361
MAE:  725.0550559822412
R2 Score:  9.113606781270963


RidgeRegression
Model training performance:
RMSE  1196.8307856345223
MAE:  723.6370749375708
R2 Score:  9.113551825604016


ElasticNet
Model training performance:
RMSE  1632.539414426596
MAE:  1135.0835947511694
R2 Score:  8.35064099195437




In [105]:
model_list

['LinearRegression', 'LassoRegression', 'RidgeRegression', 'ElasticNet']